In [106]:
metadata_pipeline_type_mappings = {'YHROHPM04': 'PEIMS_FALL_AND_PEIMS_EXYR', # FIXME: MAPPING CONFLICT
                       'YHROHPM07': 'PEIMS_FALL',
                       'YHROHPM08': 'PEIMS_FALL',
                       'YHROHPM09': 'PEIMS_FALL',
                       'YHROHPM10': 'PEIMS_FALL',
                       'YFMOHPEIM': 'PEIMS_FALL',
                       'YFIOHPEIM': 'PEIMS_MIDYR',
                       # 'YHROHPM04': 'PEIMS_EXYR', # FIXME: MAPPING CONFLICT
                       'YHROHPM01': 'TSDS_CLASS_ROSTER_WINTER',
                       'YHROHPM06': 'TSDS_CLASS_ROSTER_WINTER', # FIXME: NOT SURE (ASSUMED)
                       'YHROHPM03': 'TSDS_ECDS_KG',
                       'YHROHPM02': 'TSDS_ECDS_PK',
                       'YHROHPM05': 'TSDS_CLASS_ROSTER_FALL'}

In [107]:
# NOTE: Under Experimentation
pipeline_suffix_mappings = {'peims-submissions': '_peims',
                            'tsds_crf-submissions': '_peims',
                            'analytics': '_analytics'}

In [ ]:
# FIXME: 2024-03-21 To be parameterized
db_password_secret_name = ""
db_user_secret_name = ""
database_name = "sdb_prod_s2r_sap_peims"
data_source = "sdb_prod_s2r_sap_peims_data_source"
server_name = ""
driver = "ODBC Driver 18 for SQL Server"
semantic_schema_name = "semantic"

In [108]:
instance = instanceId = InstanceId
apiUrl = ApiUrl
schoolYear = SchoolYear
DistrictId = DistrictID = districtId = districtID
apiLimit = batchLimit

prepareSAPMetaData = prepareSAPMetadata
zone = submissionsType = sap_pipeline

### Pre-Requisites (Dev)

In [109]:
from notebookutils import mssparkutils
import configparser

config_path = "/tmp/conf.ini"
def copy_config_to_temp():
    mssparkutils.fs.cp(oea.to_url("stage1/Transactional/SAP/metadata-assets/edfi-configs.ini"),"file:/tmp/conf.ini")

def read_edfi_credentials(config_path):
    config = configparser.ConfigParser()
    config.read(config_path)

    edfi_credentials = {}

    if 'EdFi' in config:
        edfi_credentials['client_id'] = config['EdFi'].get('client_id', '')
        edfi_credentials['client_secret'] = config['EdFi'].get('client_secret', '')

    return edfi_credentials

### Actual Code

In [110]:
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StringType, StructType, StructField, IntegerType

from pyspark.sql.functions import col, substring, regexp_extract, split, lit, struct, to_date, from_unixtime, date_format
from pyspark.sql.functions import create_map, lit, when, array, coalesce, concat_ws
from pyspark.sql.functions import collect_list, create_map, lit, struct, array, concat
from pyspark.sql.functions import expr

from pyspark.sql import functions as F
import pyspark.sql.functions as f
from pyspark.sql.functions import udf

import json
import os
import pandas as pd
import re

import copy
from itertools import chain

In [111]:
print('REFINEMENT - TESTING PARAMETERIZATION')
try:
    print(kVName)
    print(workspace)
    print(apiUrl)
    print(instanceId)
    print(moduleName)
    print(apiLimit)
    print(minChangeVer)
    print(maxChangeVer)
    print(sapVersion)
    print(prepareSAPMetaData)
    print(submissions)
    print(submissionsType)
    print(schoolYear)
    print(districtID)
    print(pipelineExecutionId)

    kvName = kVName
    districtId = districtID
    districtId = districtId
except Exception as params_error:
    print('CATCHING ERROR!!!')
    print(params_error)

### Global Initializations

In [112]:
def assign_default_variable(variable_name, default_value):
    if variable_name not in globals():
        globals()[variable_name] = default_value
        logger.info(f'{variable_name} not found - using system default')

### URLs Initializations

In [113]:
%run OEA/modules/Ed-Fi/v0.7/src/utilities/edfi_v0_7_fetch_urls

In [114]:
instance_id = instanceId
school_year = schoolYear
api_year = school_year
api_url = apiUrl

# FIXME: 2024-01-31 TEMP FIX FOR FY
try:
    edfi_api_manager = EdFiApiManager(api_url, instance_id, api_year)
    edfi_api_manager.update_urls()
    edfi_api_manager.set_other_metadata()

    dependenciesUrl = edfi_api_manager.dependencies_url
    openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
    dataManagementUrl = edfi_api_manager.data_management_url
    authUrl = edfi_api_manager.auth_url

    changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
    changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
    swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

    apiVersion = edfi_api_manager.api_version
    apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion
except Exception as error:
    edfi_api_manager = EdFiApiManager(api_url, instance_id, '')
    edfi_api_manager.update_urls()
    edfi_api_manager.set_other_metadata()

    dependenciesUrl = edfi_api_manager.dependencies_url
    openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
    dataManagementUrl = edfi_api_manager.data_management_url
    authUrl = edfi_api_manager.auth_url

    changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
    changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
    swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

    apiVersion = edfi_api_manager.api_version
    apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion

### OEA Initiliazations

In [115]:
%run EdGraph/modules/SAP_PEIMS/v0.6/src/utilities/sap_peim_v0_6_sap_py

In [116]:
from datetime import datetime
oea = SAPEdFiOEAChild()   
oea.set_workspace(workspace)

In [117]:
# swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Descriptors')
oea_utils = schema_gen = SAPOpenAPIUtilChild(swagger_url)
oea_utils.create_definitions()
schemas = schema_gen.create_spark_schemas()

### SQL Connection Initializations

In [118]:
assign_default_variable(variable_name = 'server_name', 
                        default_value = 'syn-oea-hisdprod-ondemand.sql.azuresynapse.net')
assign_default_variable(variable_name = 'database_name', 
                        default_value = 'sdb_prod_s2r_sap_peims')
assign_default_variable(variable_name = 'driver', 
                        default_value = 'ODBC Driver 18 for SQL Server')

user_name = oea._get_secret(db_user_secret_name)
db_password = oea._get_secret(db_password_secret_name)
password = db_password

### Metadata for Processing

In [119]:
metadata_path = "stage1/Transactional/SAP/metadata-assets/sap-to-edfi.json"
metadata_url = oea.to_url(metadata_path)

In [120]:
jsonDF = spark.read.option("multiline", "true").json(metadata_url).cache()

json_string = jsonDF.toJSON().collect()[0]
config_data = json.loads(json_string)

### SAP & Error Logging Initiliazations

In [121]:
error_logger = ErrorLogging(spark = spark, 
                            oea = oea, 
                            logger = logger)

In [122]:
sap_utilities = SAPUtilities(spark = spark, 
                             oea = oea)

In [123]:
# sap_pipeline = "peims-submissions"

sap_to_edfi_complex = config_data.get('sap_to_edfi_complex', {})
final_columns = config_data.get('final_columns', {})
_ext_TX_cols = config_data.get('_ext_TX_cols', {})
descriptorsDFRef = config_data.get('descriptorsDFRef', {})
descriptors = config_data.get('descriptors', [])
metadata_pipeline_type_mappings = config_data.get('metadata_pipeline_type_mappings', {})

sap_process_client = SAPProcessClient(spark = spark, 
                                      oea = oea,
                                      sap_utilities = sap_utilities,
                                      sap_to_edfi_complex = sap_to_edfi_complex,
                                      final_columns = final_columns,
                                      _ext_TX_cols = _ext_TX_cols, 
                                      descriptorsDFRef = descriptorsDFRef,
                                      descriptors = descriptors) 

### SQL DB Creation - Class Definitions

In [124]:
import pyodbc
import os
import json
import logging
from datetime import datetime

class SQLDatabase:
    def __init__(self, server_name, database_name, user_name, password, driver=driver):
        self.server_name = server_name
        self.database_name = database_name
        self.user_name = user_name
        self.password = password
        self.driver = driver
        self.connection = self.connect_to_database()

    def connect_to_database(self):
        connection_string = f"DRIVER={{{self.driver}}};SERVER={self.server_name};DATABASE={self.database_name};UID={self.user_name};PWD={self.password};"
        return pyodbc.connect(connection_string)

    def execute_query(self, query, isResult = False):
        cursor = self.connection.cursor()
        cursor.execute(query)
        if isResult:
            result = int(cursor.fetchone()[0])
        else:
            result = None
        self.connection.commit()
        cursor.close()
        return result

    def close_connection(self):
        self.connection.close()

class ViewManager:
    def __init__(self, sql_db, data_source):
        self.sql_db = sql_db
        self.data_source = data_source
        self.table_suffix = ''

    def drop_view(self, schema, view_name, table_suffix):
        query = f"DROP VIEW IF EXISTS [{schema}].[{view_name}{table_suffix}]"
        self.sql_db.execute_query(query)
    
    def create_schema_if_not_exists(self, schema):
        # NOTE: Under Dev & Review
        query = f"SELECT count(*) FROM sys.schemas WHERE name = N'{schema}';"        
        isSchema = self.sql_db.execute_query(query, isResult = True)
        if isSchema == 0:
            logger.info(f'CREATING SCHEMA {schema}')
            query = f"CREATE SCHEMA [{schema}] AUTHORIZATION [dbo]"
            self.sql_db.execute_query(query)
                
    def create_view(self, schema, view_name,file_path, table_suffix):
        if file_path.startswith('stage2/Refined/'):
            file_path = "/".join(file_path.split('/')[2:])
        query = f"""CREATE VIEW [{schema}].[{view_name}{table_suffix}]
                AS SELECT *
                FROM  
                    OPENROWSET(
                        BULK '{file_path}',
                        DATA_SOURCE = '{self.data_source}',
                        FORMAT='DELTA'
                    ) nyc"""
        self.sql_db.execute_query(query)

    def add_view_to_sql_db(self, schema, view_name, file_path, overwrite=True):
        table_suffix = self.table_suffix
        if view_name.lower().endswith('exts'):
            table_suffix = '_tx'
        if overwrite:
            logging.info('Overwrite Mode Active - Dropping SQL DB Table')
            self.drop_view(schema, view_name, table_suffix)
        else:
            logging.info('Overwrite Mode Inactive')
        self.create_view(schema, view_name,file_path, table_suffix)

    def create_and_populate_views(self, schema, schema_path, view_names, table_suffix = '',overwrite=False):
        self.table_suffix = table_suffix
        self.create_schema_if_not_exists(schema)
        for view_name in view_names:
            logging.info(f'Creating View - {view_name}')
            view_path = f"{schema_path}/{view_name}"
            try:
                self.add_view_to_sql_db(schema, view_name, view_path, overwrite)
            except Exception as error:
                logging.error(f'An error occurred while creating the SQL DB Table - {error}')

### Empty Schema - Utilities

In [125]:
def initialize_sap_pipeline_vars(sapVersion = '1.0',
                                 districtId = '101912',
                                 schoolYear = '2023',
                                 base_path = '',
                                 column_mapping_file_path = ''):
    for descriptor in sap_process_client.descriptors:
        descriptor_path = f"{base_path}/ed-fi/{descriptor}"
        # FIXME: Temp Fix
        try:
            sap_process_client.descriptorsDFRef[descriptor] = sap_utilities.loadDescriptors(descriptor_path).cache()
        except Exception as error:
            sap_process_client.descriptorsDFRef[descriptor] = sap_utilities.loadDescriptors(descriptor_path.replace('/ed-fi', '/TX')).cache()

    column_mapping_file_url = oea.to_url(column_mapping_file_path) 
    column_mappings = sap_utilities.extract_refined_cols_mapping(column_mapping_file_url)

    return column_mappings

def get_non_ext_entities(entities_meta_info):
    non_ext_table_names = list()
    for entity_meta_info in entities_meta_info:
        non_ext_table_names.append(entity_meta_info['resource'].split('/')[-1])
    return non_ext_table_names

def non_common_elements(emptyTables, etlTables):
        emptyNonEtlTables = set(emptyTables) - set(etlTables) 
        emptyNonEtlTables = list(emptyNonEtlTables) #+ list(unique_in_list2)
        return emptyNonEtlTables

### Automated Views Creations

In [126]:
assign_default_variable(variable_name = 'data_source', 
                        default_value = 'sdb_prod_s2r_sap_peims_data_source')

In [127]:
sql_db = SQLDatabase(server_name, database_name, user_name, db_password)
view_manager = ViewManager(sql_db, data_source)

#### Main Tables + TX Extended Tables

In [63]:
root_path = f"stage2/Refined/SAP/analytics/pipeline={sap_pipeline}/pipelineType={sap_pipelineType}/{sapVersion}/general"
edfi_tables_path = f"{root_path}/ed-fi"
tx_tables_path = f"{root_path}/tx"

edfi_tables = oea.get_folders(edfi_tables_path)
tx_tables = oea.get_folders(tx_tables_path)
view_manager.create_and_populate_views(schema =  'dbo', 
                                       schema_path = edfi_tables_path, 
                                       view_names = edfi_tables, 
                                       table_suffix = '',
                                       overwrite = True)
view_manager.create_and_populate_views(schema =  'dbo', 
                                       schema_path = tx_tables_path, 
                                       view_names = tx_tables, 
                                       table_suffix = '_tx',
                                       overwrite = True)

#### Descriptor Tables

In [128]:
edfi_descriptorTables_path = f'stage2/Refined/SAP/analytics/descriptorTables/1.0/ed-fi/general'
tx_descriptorTables_path = f'stage2/Refined/SAP/analytics/descriptorTables/1.0/TX/general'

edfi_descriptorTables = oea.get_folders(edfi_descriptorTables_path)
tx_descriptorTables = oea.get_folders(tx_descriptorTables_path)

view_manager.create_and_populate_views(schema =  'dbo', 
                                       schema_path = edfi_descriptorTables_path, 
                                       view_names = edfi_descriptorTables, 
                                       table_suffix = '',
                                       overwrite = True)
view_manager.create_and_populate_views(schema =  'dbo', 
                                       schema_path = tx_descriptorTables_path, 
                                       view_names = tx_descriptorTables, 
                                       table_suffix = '_tx',
                                       overwrite = True)

#### Empty Tables

In [ ]:
edfi_mainTables = oea.get_folders(f"stage2/Refined/SAP/analytics/pipeline={sap_pipeline}/pipelineType={sap_pipelineType}/{sapVersion}/general/ed-fi")
tx_mainTables = oea.get_folders(f"stage2/Refined/SAP/analytics/pipeline={sap_pipeline}/pipelineType={sap_pipelineType}/{sapVersion}/general/tx")
mainTables = edfi_mainTables + tx_mainTables
        
txDescriptorTables = [sap_to_edfi_complex.get(item, item) for item in oea.get_folders(f"stage2/Ingested/SAP/descriptorTables/{sapVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/TX") if item != 'descriptorTables']
edfiDescriptorTables = [sap_to_edfi_complex.get(item, item) for item in oea.get_folders(f"stage2/Ingested/SAP/descriptorTables/{sapVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/ed-fi") if item != 'descriptorTables']
tpdmDescriptorTables = [sap_to_edfi_complex.get(item, item) for item in oea.get_folders(f"stage2/Ingested/SAP/descriptorTables/{sapVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/tpdm") if item != 'descriptorTables']
descriptorTables = txDescriptorTables + edfiDescriptorTables + tpdmDescriptorTables

In [ ]:
edfi_emptyTables = oea.get_folders('stage2/Refined/SAP/analytics/emptySchemas/general/ed-fi')
edfi_emptyTables = non_common_elements(edfi_emptyTables, 
                                        mainTables + descriptorTables)
tx_emptyTables = oea.get_folders('stage2/Refined/SAP/analytics/emptySchemas/general/tx')
tx_emptyTables = non_common_elements(tx_emptyTables, 
                                    mainTables + descriptorTables)

edfi_emptyTables_path = f'stage2/Refined/SAP/analytics/emptySchemas/general/ed-fi'
tx_emptyTables_path = f'stage2/Refined/SAP/analytics/emptySchemas/general/tx'

view_manager.create_and_populate_views(schema =  'dbo', 
                                       schema_path = edfi_emptyTables_path, 
                                       view_names = edfi_emptyTables, 
                                       table_suffix = '',
                                       overwrite = True)
view_manager.create_and_populate_views(schema =  'dbo', 
                                       schema_path = tx_emptyTables_path, 
                                       view_names = tx_emptyTables, 
                                       table_suffix = '_tx',
                                       overwrite = True)